In [21]:
from __future__ import division
from tkinter import filedialog
import numpy
from numpy.random import *

#Kmeans ++ Centroid Initialization
import random
def initializeCentriods(Data, K):
    #Taking the random data point as the first centroid using a uniformly random variable
    Centroids = [Data[random.randint(0,len(Data)-1)]]
    #Finding the next K-1 centroids using a probability distribution of distance between data point and the centroid
    #with minimum distance
    for _ in range(K-1):
        DistanceSq = numpy.array([min([numpy.inner(numpy.array(Centroid)-numpy.array(row),numpy.array(Centroid)-numpy.array(row))**0.5 for Centroid in Centroids]) for row in Data])
        individualProbabilities = DistanceSq/DistanceSq.sum()
        cumulativeprobabilities= individualProbabilities.cumsum()
        randVar = rand()
        FinalIndex=-1
        for index,probability in enumerate(cumulativeprobabilities):
            if randVar<probability:
                FinalIndex=index
                break
        Centroids.append(Data[numpy.argmax(individualProbabilities)])
    return Centroids

#Reading the content from the file and separting the attributes and the class variable C
def readfile(filename):
  lines=[line for line in open(filename)]
  lines.pop(0)
  rownames=[]
  data=[]
  classVar=[]
  for line in lines:
    p=line.strip().replace("?","0").split(',')
    rownames.append(p[0])
    data.append([float(x) for x in p[1:len(p)-1]])
    #data.append([float(x) for x in p])
    classVar.append(int(p[9]))
  return rownames,data,classVar

#For finding the Total Error rate of all the Clusters created
def totalErrorRate(CentriodIndex,classVar,K):
    CentError=[]
    TotalError=0
    #A Dictionary of count of benign and malignant patients is initialized along with
    #the type of the cluster and the error
    for i in range(K):
        CentError.append({'2':0,'4':0,'type':0,'error':0})
    #Count of benign and malignant patients is updated for each cluster
    for i in range(numRows):
        if classVar[i]==2:
            CentError[CentriodIndex[i]]['2'] +=1
        else:
            CentError[CentriodIndex[i]]['4'] +=1
    #Type and Error rate of each cluster is identified    
    for i in range(K):
        Total=CentError[i]['2']+CentError[i]['4']
        CentError[i]['type']= 2 if CentError[i]['2']>=CentError[i]['4'] else 4
        CentError[i]['error']= 1.00*CentError[i]['4']/Total if CentError[i]['2']>=CentError[i]['4'] else 1.00*CentError[i]['2']/Total
        TotalError+=CentError[i]['error']
    return CentError,TotalError

#For getting the file
Filename=filedialog.askopenfile(mode='r')
rows,data,classVar=readfile(Filename.name)
k = Filename.name.rfind("/")
Path=Filename.name[:k+1]
numRows,numCols=numpy.array(data).shape

#Number of rows and columns in the data
print (numRows,numCols)

#Number of iterations if the threshold condition is not satisfied is set to 10
itermax=10
loopiter=0
outputFile=open(Path+"KmeansOutput.csv","w")
outputFile.write("K,Iteration,BestTauLimit,ErrorRate\n")
BestCentroidIndex=[]
#Algorithm is run for K=2...5 20 Iterations each and for threshold limit 0 to 4
for K in range(100,200):
    for ITER in range(1,2):
        BestError=9999999999999
        for tau in range (0,1):

            Centroids = initializeCentriods(data,K)
            loopiter=0
            while True:
                less2Count=0
                loopiter=loopiter+1
                CentriodIndex=[]
                CentroidValue=[]
                for row in data: 
                    CentroidDiff = [numpy.inner(numpy.array(centroid)-numpy.array(row),numpy.array(centroid)-numpy.array(row))**0.5 for centroid in Centroids]
                    min_value,min_index = (min(CentroidDiff),CentroidDiff.index(min(CentroidDiff)))
                    CentriodIndex.append(min_index)
                    CentroidValue.append(min_value)
                    
                CentriodIndex=numpy.array(CentriodIndex)
                dataIndex=numpy.array(range(numRows))
                newCentroids=numpy.empty([K, numCols])
                for i in range(K):
                    sumCentroid=numpy.array([float(0)]*numCols)
                    num=dataIndex[CentriodIndex==i]
                    for index in num:
                        row=numpy.array(data[index])
                        sumCentroid+=row
                    if(len(num)<2):
                        less2Count+=1
                    if(len(num)==0): print("Empty Cluster")
                    sumCentroid=[x/len(num) for x in sumCentroid]
                    newCentroids[i]=sumCentroid
    
                thresh=Centroids-newCentroids
                taucheck=(1/K)*sum([(numpy.inner(x,x))**0.5 for x in thresh])
                CentError,error=totalErrorRate(CentriodIndex,classVar,K)
                if taucheck<=tau or loopiter>=itermax:
                    #print ("K : ",K, "Iteration : ", loopiter ,"OverallIteration : ",ITER, "Tau : " ,tau, "Error : ",error)
                    break
                else:
                    Centroids=newCentroids
                print(less2Count)
            CentError,error=totalErrorRate(CentriodIndex,classVar,K)
            
            #To get the best error among the different threshold limits tau for a particular Iteration
            if BestError>error:
                BestError=error
                BestCentroidIndex=CentriodIndex
                BestK=K
                BestTau=tau
                BestCentError=CentError
                BestCentroids=newCentroids
        print ("K : ",K,"OverallIteration : ",ITER, "Best Tau : " ,BestTau, "Error : ",BestError)
        #print(BestCentError)
        outputFile.write(str(K)+","+str(ITER)+","+str(BestTau)+","+str(BestError)+"\n")

outputFile.close()

645 8
38
38
38
38
38
K :  100 OverallIteration :  1 Best Tau :  0 Error :  3.4098214285714294
39
39
39
39
K :  101 OverallIteration :  1 Best Tau :  0 Error :  2.9817732581883525
41
41
41
41
41
41
K :  102 OverallIteration :  1 Best Tau :  0 Error :  2.685554029304029
43
43
43
43
43
43
43
43
43
K :  103 OverallIteration :  1 Best Tau :  0 Error :  2.8832722832722837
43
43
43
43
43
43
43
43
43
K :  104 OverallIteration :  1 Best Tau :  0 Error :  2.8832722832722837
42
42
42
42
42
42
K :  105 OverallIteration :  1 Best Tau :  0 Error :  3.5720728291316526
45
45
45
45
45
45
K :  106 OverallIteration :  1 Best Tau :  0 Error :  3.5066666666666664
43
43
43
43
43
43
43
43
43
K :  107 OverallIteration :  1 Best Tau :  0 Error :  3.6427750410509034
42
42
42
42
42
42
42
42
42
K :  108 OverallIteration :  1 Best Tau :  0 Error :  2.77524052065648
45
45
45
45
45
45
45
45
K :  109 OverallIteration :  1 Best Tau :  0 Error :  2.8322340813720124
44
44
44
44
44
44
44
K :  110 OverallIteration :  1 Be

In [100]:
print (BestCentroids)
print (BestCentError)
print ("Best K : ",BestK)
print ("Error Rate : " ,BestError)
print ("Best Tau : ", BestTau)

[[ 3.04301075  1.30107527  1.44301075  1.33763441  2.08817204  1.2688172
   2.10322581  1.2516129   1.10967742]
 [ 7.14957265  6.77777778  6.71367521  5.72649573  5.45726496  7.82478632
   6.08974359  6.07692308  2.54273504]]
[{'type': 2, 'error': 0.03870967741935484, '4': 18, '2': 447}, {'type': 4, 'error': 0.04700854700854701, '4': 223, '2': 11}]
Best K :  2
Error Rate :  0.08571822442790185
Best Tau :  0


[[5.0, 1.0, 1.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0], [2.0, 7.0, 10.0, 10.0, 7.0, 10.0, 4.0, 9.0, 4.0]]
[[ 3.95389987  2.37105701  2.4611499   2.14855413  2.73796688  2.58437978
   2.90269509  2.17206628  1.36195135]
 [ 5.45969474  4.84940248  4.88388013  4.2856803   4.28991295  5.43839086
   4.63973984  4.4279241   2.10037719]]
7.32594246882


In [206]:

itermax=10
loopiter=0
BestError=9999999999999
BestCentroidIndex=[]
for tau in range(5):
    for K in range(2,6):
        CentroidMembership=[]
        Centroids = initializeCentriods(data,K)
        print (Centroids)
        loopiter=0
        while True:
            loopiter=loopiter+1
            
            for row in data: 
                CentroidDiff = [numpy.inner(numpy.array(centroid)-numpy.array(row),numpy.array(centroid)-numpy.array(row))**0.5 for centroid in Centroids]
                CentroidMem = [1/(x+1) for x in CentroidDiff]
                CentroidMembership.append(CentroidMem/sum(CentroidMem))
            newCentroids=numpy.empty([K, numCols])
            membershipCount=0.0
            for i in range(K):
                sumCentroid=numpy.array([float(0)]*numCols)
                for j in range(numRows):
                    sumCentroid+=[x*CentroidMembership[j][i] for x in data[j]]
                    membershipCount+=CentroidMembership[j][i]
                newCentroids[i]=sumCentroid/membershipCount
            thresh=Centroids-newCentroids
            taucheck=(1/K)*sum([(numpy.inner(x,x))**0.5 for x in thresh])
            CentriodIndex=[]
            if taucheck<=tau or loopiter>=itermax:
                for x in CentroidMembership:
                    maxMem=0
                    maxindex=0
                    for i in range(len(x)):
                        if x[i]>maxMem:
                            maxMem=x[i]
                            maxindex=i
                    CentriodIndex.append(maxindex)
                CentError,error=totalErrorRate(CentriodIndex,classVar,K)
                if BestError>error:
                    BestError=error
                    BestCentroidIndex=CentriodIndex
                    BestK=K
                    BestTau=tau
                    BestCentError=CentError
                    BestCentroids=newCentroids
                print ("K : ",K, "Convering Iteration : ", loopiter ,"Tau : ", tau, "Threshold : " ,taucheck, "Error : ",error)
                break
            else:
                Centroids=newCentroids

[[7.0, 10.0, 10.0, 10.0, 10.0, 4.0, 10.0, 3.0], [1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 8.0]]
K :  2 Convering Iteration :  2 Tau :  0 Threshold :  0.0 Error :  0.10834094911174018
[[5.0, 7.0, 1.0, 5.0, 8.0, 3.0, 4.0, 1.0], [5.0, 10.0, 10.0, 10.0, 2.0, 10.0, 10.0, 10.0], [10.0, 3.0, 10.0, 4.0, 10.0, 10.0, 1.0, 1.0]]
K :  3 Convering Iteration :  2 Tau :  0 Threshold :  0.0 Error :  0.294822695035461
[[1.0, 2.0, 1.0, 3.0, 4.0, 1.0, 1.0, 1.0], [9.0, 10.0, 10.0, 10.0, 5.0, 10.0, 10.0, 10.0], [10.0, 3.0, 10.0, 4.0, 10.0, 10.0, 1.0, 1.0], [10.0, 5.0, 1.0, 10.0, 5.0, 3.0, 10.0, 2.0]]
K :  4 Convering Iteration :  2 Tau :  0 Threshold :  0.0 Error :  0.14949409334110292
[[7.0, 6.0, 3.0, 3.0, 8.0, 7.0, 4.0, 1.0], [5.0, 10.0, 10.0, 10.0, 2.0, 10.0, 10.0, 10.0], [1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 8.0], [10.0, 10.0, 10.0, 6.0, 1.0, 3.0, 1.0, 10.0], [9.0, 9.0, 2.0, 10.0, 6.0, 2.0, 9.0, 10.0]]
K :  5 Convering Iteration :  2 Tau :  0 Threshold :  0.0 Error :  0.11934731934731935
[[6.0, 3.0, 1.0, 4.0, 

In [49]:
print (BestCentroids)
print (BestCentError)
print ("Best K : ",BestK)
print ("Error Rate : " ,BestError)
print ("Best Tau : ", BestTau)

[[ 3.84540256  2.25449068  2.35714878  2.10244861  2.67962417  2.46543893
   2.80082225  2.05321603  1.3546531 ]
 [ 1.89588602  1.6559602   1.66159736  1.42806004  1.45869845  1.84665971
   1.60096089  1.5204331   0.70101834]]
[{'type': 2, 'error': 0.06485355648535565, '4': 31, '2': 447}, {'type': 4, 'error': 0.049773755656108594, '4': 210, '2': 11}]
Best K :  2
Error Rate :  0.11462731214146424
Best Tau :  4


In [196]:
numpy.zeros(shape=(5,2))

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]])